## Copy Scripts to Current Directory

In [ ]:
# !rm -r tcvae/
# !rm -r rg_sound_generation/

In [ ]:
!git clone https://github.com/TheSoundOfAIOSR/rg_sound_generation.git
!mv /content/rg_sound_generation/timbre_conditioned_vae/* .

In [ ]:
!pip install -q -r requirements.txt

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

## Import Scripts and Setup Config

In [ ]:
from tcvae import localconfig, train
import tensorflow as tf

conf = localconfig.LocalConfig()

In [ ]:
conf.dataset_dir = "/content/drive/MyDrive/the_sound_of_ai/new_data"
conf.checkpoints_dir = "/content/drive/MyDrive/the_sound_of_ai/new_data/checkpoints"

conf.use_encoder = False
conf.decoder_type = "cnn"
# conf.latent_dim = 4 # Does not matter
conf.batch_size = 8
conf.learning_rate = 2e-3
conf.model_name = "cnn_decoder_heuristics"
conf.step_log_interval = 10
# conf.best_loss = 3.349

# No KL Annealing
# conf.kl_anneal_start = 0
# conf.kl_weight = .1
# conf.kl_weight_max = 0.1

conf.reconstruction_weight = 1.

conf.save_config()

## Train

In [ ]:
train.train(conf)

!cp *.csv /content/drive/MyDrive/the_sound_of_ai/new_data/checkpoints/